# MUHAMMAD AL HAFNI | 2209106119 | POSTTEST 5

## Import Library yang akan digunakan

In [32]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Menampilkan data pada dataset yang digunakan

In [33]:
data = pd.read_csv('diabetes_prediction_dataset.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


Memisahkan dataset menjadi dua bagian. Variabel X berisi semua kolom dari dataset kecuali kolom 'diabetes', yang berfungsi sebagai fitur (input) untuk model. Sementara itu, variabel y berisi kolom 'diabetes', yang merupakan target (output) yang ingin diprediksi.

In [34]:
X = data.drop('diabetes', axis=1)
y = data['diabetes']

Memfilter dan memilih hanya kolom-kolom dalam variabel X yang memiliki tipe data numerik, yaitu float (bilangan desimal) dan int (bilangan bulat).

In [35]:
X = X.select_dtypes(include=[float, int])

Menstandarisasi fitur dalam dataset X. Dengan membuat objek StandardScaler, menghitung mean dan deviasi standar untuk setiap fitur. Kemudian, metode fit_transform() mengubah data X sehingga setiap fitur memiliki mean 0 dan deviasi standar 1, menghasilkan X_scaled.

In [36]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Membagi dataset menjadi data pelatihan dan data pengujian. Fungsi train_test_split() mengambil fitur yang telah distandarisasi (X_scaled) dan label target (y), dengan 30% dari data dialokasikan untuk pengujian dan 70% untuk pelatihan. Parameter random_state=42 memastikan bahwa pembagian data bersifat reproducible, sehingga hasilnya konsisten setiap kali kode dijalankan.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

1. tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)):
lapisan dens yang memiliki 32 neuron dan menggunakan fungsi aktivasi ReLU (Rectified Linear Unit). Parameter input_shape menentukan dimensi input, yang diambil dari jumlah fitur dalam data pelatihan (X_train.shape[1]).

2. tf.keras.layers.Dense(16, activation='relu'):
Lapisan kedua juga merupakan lapisan dens dengan 16 neuron dan fungsi aktivasi ReLU, yang membantu model belajar pola yang lebih kompleks.

3. tf.keras.layers.Dropout(0.3):
Lapisan dropout ini digunakan untuk mengurangi overfitting dengan menghilangkan 30% dari neuron secara acak selama pelatihan, sehingga model tidak terlalu bergantung pada neuron tertentu.

4. tf.keras.layers.Dense(1, activation='sigmoid'):
Lapisan terakhir adalah lapisan output yang memiliki satu neuron dan menggunakan fungsi aktivasi sigmoid. Ini cocok untuk klasifikasi biner, di mana outputnya akan berada di antara 0 dan 1, mewakili probabilitas dari kelas positif.

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer untuk klasifikasi biner
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 Menampilkan ringkasan arsitektur model jaringan saraf yang telah dibangun.

In [39]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 32)                  │             224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

Mengonfigurasi model jaringan saraf sebelum pelatihan dimulai. Fungsi loss yang dipilih adalah binary_crossentropy, yang cocok untuk klasifikasi biner, mengukur perbedaan antara probabilitas yang diprediksi dan aktual. Optimizer yang digunakan adalah adam, yang efisien dalam memperbarui bobot selama pelatihan. Selain itu, metrik yang dilacak adalah accuracy, yang menunjukkan proporsi prediksi yang benar.

In [40]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

Kelas MyCallback merupakan callback kustom yang menghentikan pelatihan jika akurasi validasi mencapai 95% atau lebih, dengan mencetak pesan yang sesuai. Sementara itu, early_stopping adalah callback bawaan yang menghentikan pelatihan jika tidak ada perbaikan dalam akurasi validasi selama 5 epoch berturut-turut, sambil mengembalikan bobot model ke keadaan terbaik yang dicapai. Kedua callback ini bertujuan untuk meningkatkan efisiensi dan efektivitas proses pelatihan dengan mencegah overfitting dan mempercepat konvergensi.

In [41]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') >= 0.95):
            print("\nAkurasi validasi telah mencapai > 95%!")
            self.model.stop_training = True

callbacks = MyCallback()

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True
)


Melatih model jaringan saraf dengan data pelatihan yang telah disiapkan. Data fitur dan label pelatihan disediakan melalui X_train dan y_train, dengan pelatihan dilakukan selama 50 epoch. Data validasi disertakan dengan validation_data=(X_test, y_test) untuk memantau kinerja model pada data yang tidak terlihat. Parameter verbose=1 mengatur tampilan kemajuan pelatihan, sementara callbacks=[early_stopping, callbacks] mengintegrasikan mekanisme penghentian awal untuk mencegah overfitting. Hasil pelatihan, termasuk metrik dan loss, akan disimpan dalam objek history untuk analisis lebih lanjut.

In [42]:
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    verbose=1,
    callbacks=[early_stopping, callbacks]
)

Epoch 1/50
2173/2188 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9202 - loss: 0.2278
Akurasi validasi telah mencapai > 95%!
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9204 - loss: 0.2273 - val_accuracy: 0.9619 - val_loss: 0.1097
Restoring model weights from the end of the best epoch: 1.


Mengevaluasi kinerja model pada data uji setelah proses pelatihan. Dengan memanggil metode evaluate, model menghitung nilai loss dan akurasi menggunakan data X_test dan label y_test. Hasil evaluasi ini kemudian dicetak, termasuk nilai loss dan akurasi dalam bentuk desimal melalui print

In [49]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')
print(f'Test loss: {score[0] * 100:.2f}%')
print(f'Test accuracy: {score[1] * 100:.2f}%')

Test loss: 0.1097424104809761
Test accuracy: 0.961899995803833
Test loss: 10.97%
Test accuracy: 96.19%


model.predict(X_test) digunakan untuk mendapatkan probabilitas prediksi dari data uji X_test. Selanjutnya, probabilitas tersebut diubah menjadi prediksi biner dengan menetapkan ambang batas 0,5, di mana nilai lebih besar dari 0,5 dianggap sebagai kelas 1 dan sebaliknya. Hasil prediksi ini kemudian dibandingkan dengan label asli menggunakan classification_report, yang mencetak metrik evaluasi seperti precision, recall, dan f1-score untuk setiap kelas.

In [47]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")

print(classification_report(y_test, y_pred))

938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     27453
           1       0.89      0.63      0.74      2547

    accuracy                           0.96     30000
   macro avg       0.93      0.81      0.86     30000
weighted avg       0.96      0.96      0.96     30000



Memberikan gambaran yang jelas tentang kinerja model, membantu dalam mengevaluasi apakah model telah memenuhi target akurasi yang ditentukan selama proses pelatihan.

In [51]:
print(f'Akurasi Training: {train_accuracy * 100:.2f}%')
print(f'Akurasi Validation: {val_accuracy * 100:.2f}%')

Akurasi Training: 97.26%
Akurasi Validation: 97.21%
